# Welcome to the School Achievement Dashboard! 
##### *Produced by Colin Howard and Luke Moffitt for CSCI 77800 -- EthiCS, Fall 2024*

#### For the best user experience, please start by the first cell in CleanDash.ipnyb

# Overview: How to best improve educational outcomes for all students.
* We wanted to establish a link between educational outcomes and relative funding and poverty levels to help inform policy making individuals about the best ways to support education.
* This is important because we are invested in ensuring a quality education for all students, regardless of their income levels.

**Ethically speaking, there is a concern that, as a society, our education system is biased and provides much better service for individuals of means when compared to folks living in poverty.**

* One side claims that all children have an equal right to education, regardless of their means.
* Another side affirms their right to provide the best education possible to them, using whatever means they have at their disposal.

## Possible Solutions
* There are no easy solutions to this problem!
* As the dashboard shows, additional funding does not necessarily produce better results.
* But there may be factors schools and policymakers can attend to to help improve student outcomes.

# Part 1. Poverty and Achievement

In [9]:
# imports

import ipywidgets as widgets
import json
import pandas as pd
import urllib.request as rq
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import statsmodels.api as sm
from IPython.display import display

In [30]:
# visualization 1

# variables to populate dropdowns
levels = {"New York City": "XN", "New York State": "NY", "United States": "NT"}

level = "NT"

level_selector1 = widgets.Dropdown(options=levels, description="Level:", value="NT")

subjects = ["Mathematics", "Reading", "Science"]

subject = "Mathematics"

subscales = {"Mathematics": "MRPCM", "Reading": "RRPCM", "Science": "SRPUV"}

subscale = "RRPCM"

years = {
    "Mathematics": [
        1990,
        1992,
        1996,
        1996,
        2000,
        2003,
        2005,
        2007,
        2009,
        2011,
        2013,
        2015,
        2017,
        2019,
        2022,
    ][::-1],
    "Reading": [
        1992,
        1994,
        1998,
        1998,
        2002,
        2003,
        2005,
        2007,
        2009,
        2011,
        2013,
        2015,
        2017,
        2019,
        2022,
    ][::-1],
    "Science": [2009, 2015, 2019][::-1],
}

year = 2022

grades = {"Mathematics": [4, 8], "Reading": [4, 8, 12], "Science": [4, 8, 12]}

grade = 8

variable = "SLUNCH3"

stattype = "MN:MN,PC:P1,PC:P2,PC:P5,PC:P7,PC:P9"

stattypes = {
    "MN:MN": "Mean",
    "PC:P1": "10th Percentile",
    "PC:P2": "25th Percentile",
    "PC:P5": "50th Percentile",
    "PC:P7": "75th Percentile",
    "PC:P9": "90th Percentile",
}

marker_colors = {
    "MN:MN": px.colors.qualitative.Prism[10],
    "PC:P1": px.colors.qualitative.Prism[7],
    "PC:P2": px.colors.qualitative.Prism[5],
    "PC:P5": px.colors.qualitative.Prism[4],
    "PC:P7": px.colors.qualitative.Prism[1],
    "PC:P9": px.colors.qualitative.Prism[0],
}

# method to build the dataframe
def build_frame1(subject="Mathematics", year=year, level=level, grade=grade):
    subscale = subscales[subject]
    response = rq.urlopen(
        f"https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data"
        f"&subject={subject.lower()}"
        f"&grade={grade}"
        f"&subscale={subscale}"
        f"&variable={variable}"
        f"&jurisdiction={level}"
        f"&stattype={stattype}"
        f"&year={year}"
    )
    response_string = response.read().decode("utf-8")
    try:
        df1 = pd.DataFrame(json.loads(response_string)["result"])
        df1.loc[df1["value"] == 999] = 0
        return df1
    except:
        return response_string

# method to build the the figure with graph objects (plotly)
def build_figure1(df1):
    traces = []
    for stat in stattypes.keys():
        trace = go.Bar(
            x=df1.loc[df1["stattype"] == stat]["varValueLabel"],
            y=df1.loc[df1["stattype"] == stat]["value"],
            name=stattypes[stat],
            marker={"color": marker_colors[stat]},
        )
        traces.append(trace)
    return traces

# create the initial frame and build the figure
df1 = build_frame1(subject="Mathematics", year="2022", grade="8")
fw1 = go.FigureWidget(
    data=build_figure1(df1),
    layout=go.Layout(
        barmode="group",
        title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}",
    ),
)
fw1.layout.xaxis.title="Eligibility for free or reduced-price lunch"
fw1.layout.yaxis.title=f"{subject} NAEP score"

def alert_dialog(message):
    button = widgets.Button(description="OK")
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            print("Alert dismissed.")
            button.close()
            output.clear_output()

    button.on_click(on_button_clicked)
    display(widgets.VBox([widgets.HTML(f"<b>{message}</b>"), button, output]))

# method to update the figure
def update_figure(change):
    changed = False
    global subject, year, level, grade
    if (
        (subject != subject_selector.value)
        or (year != year_selector.value)
        or (grade != grade_selector.value)
        or (level != level_selector1.value)
    ):
        changed = True
    subject = subject_selector.value
    year = year_selector.value
    level = level_selector1.value
    grade = grade_selector.value
    if changed:
        try:
            df1 = build_frame1(subject, year, level, grade)
            with fw1.batch_update():
                for i, stat in enumerate(stattypes):
                    fw1.data[i].x = df1.loc[df1["stattype"] == stat]["varValueLabel"]
                    fw1.data[i].y = df1.loc[df1["stattype"] == stat]["value"]
                fw1.update_layout(
                    title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}"
                )
                fw1.layout.yaxis.title=f"{subject} NAEP score"
        except:
            alert_dialog("Data not found! Please try again!")

# create the widgets
grade_selector = widgets.Dropdown(options=grades["Mathematics"], description="Grade level:", value=8)
subject_selector = widgets.Dropdown(options=subjects, description="Subject:")
year_selector = widgets.Dropdown(options=years["Mathematics"], description="Year:")

# method to update the years dropdown
# this is called when the subject is changed
def update_years(*args):
    year = year_selector.value
    grade = grade_selector.value
    year_selector.options = years[subject_selector.value]
    grade_selector.options = grades[subject_selector.value]
    try: # try to maintain the same grade and year if possible
        year_selector.value = year
        grade_selector.value = grade
    except: # if not, never mind
        pass

level_selector1.observe(update_figure, "value")
subject_selector.observe(update_years, "value")
subject_selector.observe(update_figure, "value")
year_selector.observe(update_figure, "value")
grade_selector.observe(update_figure, "value")
grade_selector.observe(update_years, "value")
container = widgets.VBox(
    [widgets.HBox([level_selector1, subject_selector, year_selector, grade_selector]), fw1]
)
container

# Part 2. Poverty and Funding

Although funding has gone up over time in most states, more funding does not always lead to better outcomes.

In [16]:
# visualization 2, funding by state over time
import numpy

df2 = pd.read_csv("data/FundingData.csv")
# different traces for each state
fw2 = go.FigureWidget()
fw2.layout.title = "State Per Pupil Funding over Time"
fw2.layout.xaxis.title = "Year"
fw2.layout.yaxis.title = "Funding [$/student]"
x = numpy.array(range(2021, 1986, -1))
colors = list(marker_colors.values())
for i, state in enumerate(df2["State Name"].unique()):
    # x is the year, y is the funding. individual bars are the state
    # y should access the single row of funding for the state
    fw2.add_scatter(
        x=x,
        y=df2.loc[df2["State Name"] == state].to_numpy()[0][1:],
        name=state,
        marker={"color": colors[i % len(colors)]},
    )
fw2

FigureWidget({
    'data': [{'marker': {'color': 'rgb(102, 102, 102)'},
              'name': 'ALABAMA',
              'type': 'scatter',
              'uid': 'aa3d1c80-e2d0-4fcf-b174-2c0eba235dd0',
              'x': array([2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
                          2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
                          1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987]),
              'y': array([11862, 10728, 10140, 10107, 9717, 9528, 9258, 9146, 9036, 8773, 8577,
                          8726, 8907, 8964, 9197, 8398, 7683, 7073, 6581, 6300, 6029, 5885, 5638,
                          5188, 4849, 4595, 4343, 4109, 3826, 3568, 3415, 3429, 3144, 3019, 2569,
                          2420], dtype=object)},
             {'marker': {'color': 'rgb(204, 80, 62)'},
              'name': 'ALASKA',
              'type': 'scatter',
              'uid': 'f70573d2-93c0-4496-9beb-1103

At both the national and state levels, funding per student tends to go down as the percentage of poverty goes up (although there are many outliers in the data). For every 10% increase in childhood poverty, a district spends on average $268 less per student. *Note that this does not take into account local cost of living, which tends to be higher in urban districts with higher rates of poverty.*

In [33]:
# visualization 3, funding vs poverty
# scatterplot that shows school poverty vs funding
# we already know that poverty impacts achievement, from previous graph
# now we want to show the correlation between poverty and funding
# we are going to do this using data from NCES and the Census Bureau

# data are not available at the city level
levels2 = {"New York State": "NY", "United States": "NT"}

level2 = "NY"

level_selector2 = widgets.Dropdown(options=levels2, description="Level:", value="NT")

# Source: https://nces.ed.gov/ccd/elsi/tableGenerator.aspx
povertyDF = pd.read_csv("data/DistrictFundingData.csv")
# Source: https://www.census.gov/data/datasets/2022/demo/saipe/2022-school-districts.html
censusDF = pd.read_csv("data/ussd22.csv")
censusDF["Agency ID - NCES Assigned [District] Latest available year"] = (
    censusDF["State FIPS Code"].astype(str).str.zfill(2)
    + censusDF["District ID"].astype(str).str.zfill(5)
).astype(np.int64)
# combine both FIPS ID with the distrcit ID from census data

df3 = pd.merge(censusDF, povertyDF)

# divide children in poverty by the total population
df3["Percentage in poverty"] = (
    df3[
        "Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder"
    ]
    .str.replace(",", "")
    .astype(int)
    / df3["Estimated Population 5-17"].str.replace(",", "").astype(int)
    * 100
)

# clean and organize data
df3 = df3[
    df3[
        "Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2021-22"
    ].str.isnumeric()
]
df3["Expenditures per student"] = df3[
    "Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2021-22"
].astype(int)
df3.dropna(subset=["Expenditures per student", "Percentage in poverty"], inplace=True)

# create the visualization
fw3 = go.FigureWidget()
fw3.layout.title = "District Per Pupil Funding vs. Childhood Poverty"
fw3.layout.xaxis.title = "Percentage of children living in poverty [%]"
fw3.layout.yaxis.title = "Expenditures [$/student]"
scatter = go.Scatter(
    x=df3["Percentage in poverty"],
    y=df3["Expenditures per student"],
    mode="markers",
    text=df3["Agency Name"],
    name="District Data",
    marker={"color": marker_colors['PC:P7']},
)
fw3.add_trace(scatter)
X = df3["Percentage in poverty"]
X = sm.add_constant(X)
model = sm.OLS(df3["Expenditures per student"], X).fit()
X_pred = sm.add_constant(np.linspace(0, 100, 100))
yModel = model.predict(X_pred)
line = go.Scatter(
    x=np.linspace(0, 100, 100), y=yModel, mode="lines", name="Line of Best Fit",
    marker={"color": marker_colors['PC:P9']},
)
fw3.add_trace(line)

# updates the figure depending on level selection
def update_poverty(change):
    level2 = level_selector2.value
    # need to restrict data based on the area

    if level2 == "NT":
        tempdf = df3
    elif level2 == "NY":
        tempdf = df3[
            df3["State Abbr [District] Latest available year"].str.contains("NY")
        ]
    elif level2 == "XN":
        # NEW YORK CITY per pupil data not available
        tempdf = df3[df3["Agency Name [District] 2021-22"].str.contains("NEW YORK CITY")]
    with fw3.batch_update():
        fw3.data[0].y = (
            sm.OLS(tempdf["Expenditures per student"], tempdf["Percentage in poverty"])
            .fit()
            .predict(np.linspace(0, 100, 100))
        )
        fw3.data[0].x = tempdf["Percentage in poverty"]
        fw3.data[0].y = tempdf["Expenditures per student"]
        fw3.data[0].text = tempdf["Agency Name"]

level_selector2.observe(update_poverty, "value")

container = widgets.VBox([widgets.HBox([level_selector2]), fw3])
container

# Part 3. Factors Affecting Achievement

Here are a select few other variables which also affect achievement. Experiment with the choices to see the results.

In [27]:
# visualization 4 (part 3)
# Use the same level and subject selector, make a new variable selector

subjects = ["Mathematics", "Reading", "Science"]

subject = "Mathematics"

subscales = {"Mathematics": "MWPCM", "Reading": "RRPCM", "Science": "SRPUV"}

subscale = "RRPCM"

year = 2019
grade = 12

variables = {
    "Percent of teachers absent on average day": "C036501",
    "School type is independent charter": "C0863J1",
    "Talk about studies at home": "B017451",
    "Use laptop or desktop computer during class": "B034701",
    "Use tablet during class": "B034801",
    "Days absent from school in the last month": "B018101",
}

factor = "C036501"
variable = "C036501"

def build_frame4(
    subject="Mathematics",
    factor=factor,
    year=year,
    level=level
):
    subscale = subscales[subject]
    url = (
        f"https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data"
        + f"&subject={subject.lower()}"
        + f"&grade=12"
        + f"&subscale={subscale}"
        + f"&variable={factor}"
        + f"&jurisdiction={level}"
        + f"&stattype={stattype}"
        + f"&year={year}"
    )
    response = rq.urlopen(url)
    response_string = response.read().decode("utf-8")
    try:
        df = pd.DataFrame(json.loads(response_string)["result"])
        return df
    except:
        return response_string

def build_figure4(df):
    traces = []
    for stat in stattypes.keys():
        trace = go.Bar(
            x=df.loc[df["stattype"] == stat]["varValueLabel"],
            y=df.loc[df["stattype"] == stat]["value"],
            name=stattypes[stat],
            marker={"color": marker_colors[stat]},
        )
        traces.append(trace)
    return traces

df4 = build_frame4()
fw4 = go.FigureWidget(
    data=build_figure4(df4),
    layout=go.Layout(
        barmode="group",
        title=f"12th Grade {subject} NAEP Scores and {dict(zip(variables.values(), variables.keys()))[factor].title()}, {year}, {dict(zip(levels.values(), levels.keys()))[level]}",
    ),
)
fw4.layout.xaxis.title = variable

def update_figure4(change):
    subject = subject_selector.value
    factor = variable_selector.value
    level = level_selector1.value
    grade = grade_selector.grade
    # what happens when the API complains?
    df = build_frame4(subject, year, level,)
    with fw4.batch_update():
        for i, stat in enumerate(stattypes):
            fw4.data[i].x = df.loc[df["stattype"] == stat]["varValueLabel"]
            fw4.data[i].y = df.loc[df["stattype"] == stat]["value"]
        variable = dict(zip(variables.values(), variables.keys()))[factor]
        fw4.update_layout(
            title=f"{grade}th Grade {subject} NAEP Scores and {variable.title()}, {year}, {dict(zip(levels.values(), levels.keys()))[level]}",
        )
        fw4.layout.xaxis.title = variable

variable_selector = widgets.Dropdown(options=variables, description="Variable:")

level_selector1.observe(update_figure4, "value")
subject_selector.observe(update_figure4, "value")
variable_selector.observe(update_figure4, "value")

container = widgets.VBox(
    [widgets.HBox([level_selector1, subject_selector, year_selector, variable_selector]), fw4]
)
container

# Conclusions
With the understanding that higher funding does not neccesarily lead to better outcomes, care must be taken to understand how the money is being spent.

One of the biggest takeaways from this project was the suprise that some schools are spending around 500,000 dollars per year on an individual student. However, even with more than 10x the funding, these schools are not performing 10x as well.

Ultimately, policy makers need to have better information about how schools budgets are broken up. Publicly available data would be more useful if it included additonal data points regarding how the money was spent. As is, it is very difficult to authoritatively discern which factors do and do not directly contribute to student outcomes.


## Summary
1. **Background on your ethical topic, including a debate on the various dimensions/sides of it.** 
   1. **What is a quick overview of this issue?** We wanted to establish a link between educational outcomes and relative funding and poverty levels to help inform policy making individuals about the best ways to support education.
   2. **Why should we be addressing this issue?  (Why is it important?)** This is important because we are invested in ensuring a quality education for all students, regardless of their income levels.
   3. **What are the ethical concerns?** The conern is that, as a society, our education system is biased and provides much better service for individuals of means when compared to folks living in poverty.
   4. **What are the ethical justifications for both sides?** While one side claims that all children have an equal right to education, the other side affirms their right to provide the best education possible to them, using whatever means they have at their disposal.
   5. **What are the possible solutions?** While there are no easy answers to this debate, the truth is that funding does not automatically lead to better results. Instead, the decisions made within school districts about how to spend the money may be more informative. 
2. **References that informed your debate.** We took information from the NAEP (the Nation's Report Card) and from the US Census Bureau to help us make our case.
3. **Introduction of your software solution.** 
   1. **How does your software solution relate to the ethical topic?** This software makes it easy for a user to look at the disparities in funding and educational outcomes in our nation at a variety of levels. It invites thought and discussion about how to best fund our schools, and encourages policy makers to think outside of the box when creating solutions.
   2. **A demonstration of how the software can be used to weigh in on aspects of the ethical topic; perhaps reinforcing the debate in #1.** *See software above!*

4. **Design of your software.** 
   1. **A deeper dive into interesting aspects of the code, the dataset, or the development process that would be educational to others learning about coding in Python, coding with data, or coding for ethics.** Plotly (`plotly`) has a bunch of cool stuff. You can also see how we had to manipulate the data in `pandas` `DataFrame`s to merge the two sets.
   2. **Show us the most complex portions of your code and explain the implementation.** See visualization #3, `function update_poverty()`